<a href="https://colab.research.google.com/github/rikideguchi/Signate-HiroshimaQuest/blob/master/%E3%81%B2%E3%82%8D%E3%81%97%E3%81%BEQuest_%E3%82%B3%E3%83%BC%E3%82%B9%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv('../content/drive/My Drive/データ解析コンペ/Signate/ひろしまQuest 野球コンペ/コース部門/train_pitch.csv')
test = pd.read_csv('../content/drive/My Drive/データ解析コンペ/Signate/ひろしまQuest 野球コンペ/コース部門/test_pitch.csv')

In [0]:
data = pd.concat([train, test], ignore_index=True)

# Feature Engineering

In [0]:
data.drop(['試合内連番', '日付', '時刻', 'ホームチームID', 'アウェイチームID', '球場ID', '球場名', '試合種別詳細', 
           '表裏', 'イニング内打席数', '投手チームID', '投手登板順', '投手試合内対戦打者数', '打者チームID', '打者守備位置', 
           '打者試合内打席数', 'プレイ前ホームチーム得点数', 'プレイ前アウェイチーム得点数', '二塁走者ID', '三塁走者ID', 
           '一塁手ID', '二塁手ID', '三塁手ID', '遊撃手ID', '左翼手ID', '中堅手ID', '右翼手ID', '打者打席左右', '試合ID', '試合内投球数', '打者打順', '成績対象打者ID', 
           '投手イニング内投球数', 'イニング', '年度', '打席内投球数', '投手役割', 'プレイ前アウト数', '成績対象投手ID','成績対象打者ID'], axis = 1, inplace = True)

In [0]:
data['プレイ前走者状況'].replace(['___', '_2_', '1__', '12_', '_23', '1_3', '__3', '123'], 
                         [0, 1, 2, 3, 4, 5, 6, 7], inplace = True)
data['プレイ前走者状況'] = data['プレイ前走者状況'].astype(np.int8)

data['投手試合内投球数'] = data['投手試合内投球数'].astype(np.int16)

data['一塁走者ID'].fillna(0, inplace = True)
data['一塁走者ID'] = data['一塁走者ID'].astype(np.int32)
data['投手投球左右'].replace(['左', '右'], [0, 1], inplace = True)
data['投手投球左右'] = data['投手投球左右'].astype(np.int8)
data['データ内連番'] = data['データ内連番'].astype(np.int32)
data['球種'] = data['球種'].astype(np.float16)
data['投手ID'] = data['投手ID'].astype(np.int32)
data['打者ID'] = data['打者ID'].astype(np.int32)
data['プレイ前ボール数'] = data['プレイ前ボール数'].astype(np.int8)
data['プレイ前ストライク数'] = data['プレイ前ストライク数'].astype(np.int8)
data['プレイ前走者状況'] = data['プレイ前走者状況'].astype(np.int8)
data['一塁走者ID'] = data['一塁走者ID'].astype(np.float32)
data['捕手ID'] = data['捕手ID'].astype(np.int32)

In [0]:
train = data[:len(train)]
test = data[len(train):]

In [0]:
y_train = train['投球位置区域']
X_train = train.drop('投球位置区域', axis = 1)
X_test = test.drop('投球位置区域', axis = 1)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size = 0.3, 
                    random_state = 0, stratify = y_train)

In [0]:
categorical_features = ['球種','投手ID', '投手投球左右','打者ID',
       'プレイ前ボール数', 'プレイ前ストライク数', 'プレイ前走者状況', '一塁走者ID', '捕手ID']

# xgboost

In [0]:
import xgboost as xgb
from sklearn.metrics import log_loss

dtrain = xgb.DMatrix(X_train, label = y_train)
dvalid = xgb.DMatrix(X_valid, label = y_valid)
dtest = xgb.DMatrix(X_test)

params = {
    'objective':'multi:softprob', 
    'eval_metric': 'mlogloss', 
    'num_class':13
}

num_round = 500

watchlist = [(dtrain, 'train'), (dvalid, 'eva')]

model = xgb.train(params, dtrain, num_round, evals = watchlist, early_stopping_rounds=10)

y_pred = model.predict(dtest)

In [0]:
sub = pd.read_csv('../content/drive/My Drive/データ解析コンペ/Signate/ひろしまQuest 野球コンペ/コース部門/sample_submit_course.csv')
sub = sub.join(pd.DataFrame(y_pred))
sub.columns = ['0', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.076, 0.076, 0.076, 0.076, 0.076, 0.076,0.076, 0.076, 0.076, 0.076, 0.076, 0.076, 0.076]
sub.to_csv('submission_xgboost.csv', index=False) 

# lightgbm

In [0]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, 
                       categorical_feature = categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train, 
                      categorical_feature=categorical_features)
params = {
    'objective': 'multiclass', 
    'metric': {'multi_logloss'},
    'num_class': 13
}

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    num_boost_round = 1000,
    verbose_eval=10,
    early_stopping_rounds = 10)

y_pred = model.predict(X_test, num_iteration = model.best_iteration)

In [0]:
sub = pd.read_csv('../content/drive/My Drive/データ解析コンペ/Signate/ひろしまQuest 野球コンペ/コース部門/sample_submit_course.csv')
sub = sub.join(pd.DataFrame(y_pred))
sub.columns = ['0', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.076, 0.076, 0.076, 0.076, 0.076, 0.076,0.076, 0.076, 0.076, 0.076, 0.076, 0.076, 0.076]
sub.to_csv('submission_lightgbm.csv', index=False) 


In [0]:
from google.colab import files
files.download('submission_xgboost.csv') 